---
title: "レポート課題"

date: "2025-02-04"
execute:
  error: True
  cache: true
  echo: True
format:
  html:
    slide-level: 2
    toc: True
---

#  [問題1] 

- 業務や身の回りの適当な問題を線形計画問題や整数計画問題(一部の変数のみに整数条件がつく問題でもよい), 2次計画問題やネットワーク最適化問題に定式化してみよ．
- もし，可能であれば，解いてみよ

## データ収集
- 日経225、S&P500、BTCに関する過去５年分の終値データ（1,223件）をYahoo! Financeから取得し、ドル/円の為替レートを用いて日本円に換算。

In [42]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Yahoo Financeからデータを取得
symbols = ["^N225", "^GSPC", "BTC-USD"]
period = "5y"  # 5年分

# 各ティッカーのデータを取得
df_close = pd.DataFrame()
for symbol in symbols:
    ticker = yf.Ticker(symbol)
    data = ticker.history(period=period)  
    data.index = data.index.date
    df_close[symbol] = data["Close"]

# 欠損値を前日のデータで補完
df_close = df_close.ffill()

# 為替レート（USD/JPY）の取得
usd_jpy_ticker = yf.Ticker("JPY=X")
usd_jpy = usd_jpy_ticker.history(period=period)["Close"]
usd_jpy.index = usd_jpy.index.tz_localize(None)
usd_jpy = usd_jpy.reindex(df_close.index, method="ffill")

# USDをJPYに変換
df_close["^GSPC/JPY"] = df_close["^GSPC"] * usd_jpy
df_close["BTC/JPY"] = df_close["BTC-USD"] * usd_jpy
df_close = df_close.drop(columns=["^GSPC", "BTC-USD"])

# 結果を確認
print(f"新しいデータの上位: {df_close.tail()}\n")
print(f"古いデータの上位: {df_close.head()}\n")

print(f"データの件数: {len(df_close)}")

新しいデータの上位:                    ^N225      ^GSPC/JPY       BTC/JPY
2025-01-29  39414.781250  939469.068054  1.613197e+07
2025-01-30  39513.968750  942063.425661  1.625178e+07
2025-01-31  39572.488281  930827.519041  1.578031e+07
2025-02-03  38520.089844  923745.226934  1.578031e+07
2025-02-04  38730.500000  929392.119758  1.530555e+07

古いデータの上位:                    ^N225      ^GSPC/JPY       BTC/JPY
2020-02-04  23084.589844  358157.853016  9.971627e+05
2020-02-05  23319.560547  364971.805336  1.052160e+06
2020-02-06  23873.589844  367413.498144  1.068468e+06
2020-02-07  23827.980469  365941.616702  1.077240e+06
2020-02-10  23685.980469  367539.906434  1.080728e+06

データの件数: 1223


## リターンとリスクの評価

In [38]:
# 日次リターンを計算
df_return = df_close.pct_change().dropna()

# 各資産の平均リターン
expected_returns = df_return.mean()

# リターンの共分散行列
cov_matrix = df_return.cov()

# 各資産のリスク（標準偏差）を計算
std_devs = np.sqrt(np.diag(cov_matrix))

print(f"日次リターンの平均:\n {expected_returns}\n")

print(f"各資産の標準偏差:\n {pd.Series(std_devs, index=df_return.columns)}\n")

print(f"共分散行列:\n {cov_matrix}\n")

日次リターンの平均:
 ^N225        0.000517
^GSPC/JPY    0.000889
BTC/JPY      0.003137
dtype: float64

各資産の標準偏差:
 ^N225        0.013711
^GSPC/JPY    0.014688
BTC/JPY      0.041959
dtype: float64

共分散行列:
               ^N225  ^GSPC/JPY   BTC/JPY
^N225      0.000188   0.000059  0.000069
^GSPC/JPY  0.000059   0.000216  0.000246
BTC/JPY    0.000069   0.000246  0.001761



## シャープレシオの最大化

In [43]:
# シャープレシオを最大化する最適化関数
def negative_sharpe_ratio(weights, expected_returns, cov_matrix):
    portfolio_return = np.sum(weights * expected_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return -(portfolio_return / portfolio_volatility)

# 最適化の実行
num_assets = len(expected_returns)
initial_guess = np.ones(num_assets) / num_assets  # 初期ウェイト（均等分割）
bounds = [(0, 1) for _ in range(num_assets)]  # 各資産のウェイト範囲
constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}  # 合計ウェイトが1になる制約

result = minimize(negative_sharpe_ratio, initial_guess, args=(expected_returns, cov_matrix), 
                  method='SLSQP', bounds=bounds, constraints=constraints)

# 最適なウェイトを取得
optimal_weights = result.x

# 投資額（3万円）の分配
investment = 30000
amount_invested = optimal_weights * investment

assets = expected_returns.index
print("最適なポートフォリオ:")
for asset, weight, amount in zip(assets, optimal_weights, amount_invested):
    print(f"資産: {asset}")
    print(f"- 最適ウェイト: {np.round(weight, 2)}")
    print(f"- 投資額: {np.round(amount, 2)}\n")

最適なポートフォリオ:
資産: ^N225
- 最適ウェイト: 0.31
- 投資額: 9321.07

資産: ^GSPC/JPY
- 最適ウェイト: 0.4
- 投資額: 12147.43

資産: BTC/JPY
- 最適ウェイト: 0.28
- 投資額: 8531.5

